In [ ]:
from pathlib import Path
import pandas as pd
import seaborn as sns
import json
import os
import matplotlib.pyplot as plt

# Für Performance: times, provider_times, output.begin/end.
#Für Kosten: billing.
#Für Zuverlässigkeit: stats.failure, stats.cold_start.
#Für Ressourcenverbrauch: memory_used.

verzeichnis = "Untitled Folder"
dateien = [f for f in os.listdir(verzeichnis) if f.endswith('.json')]

alle_daten = []
for datei in dateien:
    pfad = os.path.join(verzeichnis, datei)
    try:
        if os.path.getsize(pfad) == 0:
            print(f"⚠️ Datei ist leer: {datei}")
            continue
        with open(pfad, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"!!! Fehler beim Einlesen von {datei}: {e}")
        continue
    except Exception as e:
        print(f"!!! Unerwarteter Fehler bei {datei}: {e}")
        continue
    
    if not data:
        print(f"⚠️ Keine Daten in Datei: {datei}")
        continue
    
    # Extrahiere Metadaten aus dem Dateinamen
    teile = datei.split('_')
    if len(teile) < 5:
        print(f"⚠️ Unerwartetes Dateiformat: {datei}")
        continue
    
    anbieter = teile[0]
    region = teile[1]
    experiment = teile[2]
    aufruf = teile[3]
    
    ignored = 0
    for eintrag in data:
        if isinstance(eintrag, dict):
            if 'exec_time_ms' in eintrag or 'latency_ms' in eintrag:
                # ...wie oben...
                alle_daten.append(eintrag)
        else:
            ignored += 1
    if ignored:
        print(f"{ignored} Einträge ignoriert (kein Messwert-Dict) in Datei {datei}")

df = pd.DataFrame(alle_daten)
# Beispiel: Wenn deine Spalten exec_time_ms und latency_ms heißen:
df_clean = df.dropna(subset=['exec_time_ms', 'latency_ms'])
df_clean = df_clean[(df_clean['exec_time_ms'] >= 0) & (df_clean['latency_ms'] >= 0)]
df_clean['exec_time_ms'] = pd.to_numeric(df_clean['exec_time_ms'], errors='coerce')
df_clean['latency_ms'] = pd.to_numeric(df_clean['latency_ms'], errors='coerce')
df_clean = df_clean.dropna(subset=['exec_time_ms', 'latency_ms'])
df_clean['exec_time_ms'] = df_clean['exec_time_ms'].astype(float)
df_clean['latency_ms'] = df_clean['latency_ms'].astype(float)
# Sortieren nach 'aufruf' (hier ggf. 'start_type') und 'experiment' (hier ggf. 'scenario')
df_clean = df_clean.sort_values(by=["start_type", "scenario"])
df_clean = df_clean.reset_index(drop=True)
df_clean.head()




7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_110_cold_results_1024.json
7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_110_cold_results_2048.json
7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_110_cold_results_256.json
7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_110_cold_results_3008.json
7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_110_cold_results_512.json
7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_110_warm_results_1024.json
7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_110_warm_results_2048.json
7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_110_warm_results_256.json
7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_110_warm_results_3008.json
7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_110_warm_results_512.json
7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_120_cold_results_1024.json
7 Einträge ignoriert (kein Messwert-Dict) in Datei aws_eu_120_cold_re

KeyError: ['exec_time_ms', 'latency_ms']

NameError: name 'files' is not defined

In [ ]:
# Beispielhafte Umbenennung auf erwartete Spalten
rename_map = {
    "duration_ms": "exec_time_ms",
    "duration": "exec_time_ms",
    "latency": "latency_ms",
    "cpu_time_ms": "cpu_ms",
    "memory_gb_s": "mem_mb_s",  # ggf. skaliert konvertieren
    "transfer_mb": "data_transfer_mb"
}
df = df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns})

# Metadaten aus Dateinamen extrahieren (optional, je nach Namensschema)
def extract_meta(path: Path):
    # Beispiel: aws_eu_110_cold_results_256.json
    parts = path.stem.split("_")
    meta = {}
    if len(parts) >= 6:
        meta["provider"] = parts
        meta["scenario"] = parts[12]
        meta["load"] = int(parts[13]) if parts[13].isdigit() else parts[13]
        meta["start_type"] = parts[14]
        meta["platform"] = "lambda"  # Beispiel; ggf. aus Inhalt ableiten
        meta["settings"] = {"memory": parts[15]}
    return meta

meta_rows = []
for f in files:
    meta_rows.append(extract_meta(f))
import pandas as pd
meta_df = pd.DataFrame(meta_rows)
df[[c for c in meta_df.columns if c not in df.columns]] = meta_df[[c for c in meta_df.columns if c not in df.columns]].values


NameError: name 'df' is not defined

In [ ]:
from pathlib import Path
import pandas as pd, json

data_dir = Path(r"C:\Users\alexa\Desktop\Untitled Folder")
print("Suche in:", data_dir.resolve())

# 1) Dateien sammeln: .json und .jsonl, notfalls rekursiv
files = sorted(list(data_dir.glob("*.json"))) + sorted(list(data_dir.glob("*.jsonl")))
if not files:
    files = sorted(list(data_dir.rglob("*.json"))) + sorted(list(data_dir.rglob("*.jsonl")))
print(f"{len(files)} Dateien gefunden")
print([f.name for f in files[:5]])  # Stichprobe

def load_one(f: Path) -> pd.DataFrame:
    # Versuch A: JSON-Lines
    try:
        dfA = pd.read_json(f, lines=True)
        if not dfA.empty:
            return dfA
    except ValueError:
        pass
    # Versuch B: Normales JSON
    obj = json.loads(f.read_text(encoding="utf-8"))
    if isinstance(obj, list):
        return pd.DataFrame(obj)
    if isinstance(obj, dict):
        # Heuristik: erst Liste von Records in einem Key suchen
        for k, v in obj.items():
            if isinstance(v, list) and v and isinstance(v, dict):
                return pd.json_normalize(v)
        # Sonst Top-Level-Dict flatten
        return pd.json_normalize(obj)
    # Fallback: leere Tabelle
    return pd.DataFrame()

parts = []
for f in files:
    dfi = load_one(f)
    print(f"{f.name}: {dfi.shape}")
    parts.append(dfi)

df = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame()
print("Gesamt:", df.shape)
df.head()


In [ ]:
expected = {"provider","platform","scenario","load","start_type","exec_time_ms","latency_ms",
            "cpu_ms","mem_mb_s","data_transfer_mb","cost","settings"}
missing = expected - set(df.columns)
if missing:
    # Beispiel: fehlende Spalten mit NaN auffüllen (besser: korrekt mappen/ableiten)
    for col in missing:
        df[col] = pd.NA
# harte Prüfung
missing = expected - set(df.columns)
assert not missing, f"Fehlende Spalten: {missing}"


In [ ]:
metrics = ["exec_time_ms","latency_ms","cost"]

def iqr_filter(group, cols):
    for c in cols:
        q1 = group[c].quantile(0.25)
        q3 = group[c].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5*iqr
        upper = q3 + 1.5*iqr
        group = group[(group[c] >= lower) & (group[c] <= upper)]
    return group

df_clean = (df
    .groupby(["provider","platform","scenario","load","start_type"], group_keys=False)
    .apply(lambda g: iqr_filter(g, metrics)))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="whitegrid")
g = sns.catplot(
    data=df_clean, kind="box",
    x="provider", y="latency_ms",
    col="load", row="start_type",
    sharey=False, height=3, aspect=1.2
)
g.set_xticklabels(rotation=30)
plt.show()
